In [ ]:
import os
import yaml
import pandas as pd
import shutil

mlruns_path = "/home/work/model/multi_input_model/mlruns"

def read_yaml_file(file_path):
    with open(file_path, 'r') as file:
        try:
            data = yaml.safe_load(file)
            return data
        except yaml.YAMLError as e:
            print(f'YAML 파일을 읽는 중 오류가 발생했습니다: {e}')

# mlruns에 있는 모든 run을 돌면서 delete상태인 run을 실제로 삭제함.
def convert_all():
    for experiment in os.listdir(mlruns_path):
        if not experiment.endswith('.yaml') and not experiment.startswith('.'):
            experiment_path = os.path.join(mlruns_path, experiment)
            convert_experiment(experiment_path)

# expriment안의 모든 runs
def convert_experiment(experiment_path):
    for run in os.listdir(experiment_path):
        if not run.endswith('.yaml'):
            run_path = os.path.join(experiment_path, run)
            try:
                convert_run(run_path)
            except:
                pass

# 하나의 run
def convert_run(run_path):
    yaml_path = os.path.join(run_path, "meta.yaml")
    data = read_yaml_file(yaml_path)
    try:
        lifecycle_stage = data['lifecycle_stage']
        if lifecycle_stage == 'deleted':
            shutil.rmtree(run_path)
            print(run_path,"deleted")
        else:
            print(run_path,"active")
    except:
        shutil.rmtree(run_path)
        print(run_path,"deleted")

convert_all()

/home/work/model/multi_input_model/mlruns/23/994cef8bdb6c47ebb49270559a35a782 active
/home/work/model/multi_input_model/mlruns/23/ea391ec1e4e34783b2553c3534190515 active
/home/work/model/multi_input_model/mlruns/23/66eb01b181504fb9b4caef9c8d2b3726 active
/home/work/model/multi_input_model/mlruns/23/5d99299128f242999a45e53b280d88ff active
/home/work/model/multi_input_model/mlruns/23/c04322a540b54376a21223dcce756160 active
/home/work/model/multi_input_model/mlruns/23/f66a7e2303804739a312bb3f6f4cf8f5 active
/home/work/model/multi_input_model/mlruns/23/64dd1d76d2794327b40dd7c6366da37f active
/home/work/model/multi_input_model/mlruns/46/35e412a2049c408da85ccabc2911d147 active
/home/work/model/multi_input_model/mlruns/46/6013f1887ad0401daafa2d381f462d04 active
/home/work/model/multi_input_model/mlruns/46/703c5d6585174ea190ee4c1ed7ebbd5b active
/home/work/model/multi_input_model/mlruns/53/b1c8a8288c014aa2ab3653d0755d741a active
/home/work/model/multi_input_model/mlruns/54/2e2be632981048afafc1

/home/work/model/multi_input_model/mlruns/40/06ae7315a3844f039dd73d9d03e8f0c4 deleted
/home/work/model/multi_input_model/mlruns/40/2bc1da09da224148b5123ded008f9d1b deleted
/home/work/model/multi_input_model/mlruns/40/d7020bed17e640e3b376371ebb7ec69b deleted
/home/work/model/multi_input_model/mlruns/40/12b26fb7d3e2421fa10185d58992c65b active
/home/work/model/multi_input_model/mlruns/40/5a4fe6c5e5cf45d3ab7b054de977cf94 active
/home/work/model/multi_input_model/mlruns/40/0b4f27f892914873873bea44642b1e7c active
/home/work/model/multi_input_model/mlruns/40/cff0e151e4894554b43816ce9d3f3959 deleted
/home/work/model/multi_input_model/mlruns/40/a03ffdd4f981439b94f8187cb8beb4e6 active
/home/work/model/multi_input_model/mlruns/40/e4478c9fa17f4cda8d18b8451c205a6c active
/home/work/model/multi_input_model/mlruns/40/e75a4a26e1a649edb70ec16f28fdc1a9 deleted
/home/work/model/multi_input_model/mlruns/40/7aeca36782844a7ba8f70d41074cc302 deleted
/home/work/model/multi_input_model/mlruns/40/60a4eb036fc649

/home/work/model/multi_input_model/mlruns/14/11272ab74ad045a3aa096f937f1ced43 deleted
/home/work/model/multi_input_model/mlruns/47/1f80c14e57534eae9c28c95230595864 active
/home/work/model/multi_input_model/mlruns/4/c684c40237c24070b20fa474deeada98 active
/home/work/model/multi_input_model/mlruns/24/6f1e310433ca4f65aac96905ef48be20 active
/home/work/model/multi_input_model/mlruns/24/4f5c7670157642e0ad0cfb1642178a0d active
/home/work/model/multi_input_model/mlruns/24/2dce5639578e4093b68c022f02edffa8 active
/home/work/model/multi_input_model/mlruns/24/3cd105ebe06d44219988897d7d3e31aa active
/home/work/model/multi_input_model/mlruns/24/640b88efa0dd4ef0ad8571b1d4699611 active
/home/work/model/multi_input_model/mlruns/24/561c805bb28c4819b8c7560d5815eafe active
/home/work/model/multi_input_model/mlruns/27/cfd7a8eb5b344a8c8c42e8148eca3c0f active
/home/work/model/multi_input_model/mlruns/33/14747517d9b345b48e7171ee25646d35 active
/home/work/model/multi_input_model/mlruns/33/2ea95612c4dd40eea64b

/home/work/model/multi_input_model/mlruns/21/7a4160286f0b4cf1a747be42027bdd4f deleted
/home/work/model/multi_input_model/mlruns/21/edd9d972ed3946acae496e92f6d0e1ff deleted
/home/work/model/multi_input_model/mlruns/21/8b041f90b3d545c8a48ab7676ff1e134 deleted
/home/work/model/multi_input_model/mlruns/21/3bcca7a15d2e4f7c8007fe8b1ba8d2bf deleted
/home/work/model/multi_input_model/mlruns/21/35f7d4e978b440b7bcfd00f8c61834cc active
/home/work/model/multi_input_model/mlruns/21/908702d117984e1187dad1e4a542f528 deleted
/home/work/model/multi_input_model/mlruns/21/2fa47931a0744b8591d6a995e2d69935 deleted
/home/work/model/multi_input_model/mlruns/21/e2d9928c86a8403fa4a27e7cbddecdda deleted
/home/work/model/multi_input_model/mlruns/21/9d428d06afd04fdd8850203e5b383b39 deleted
/home/work/model/multi_input_model/mlruns/21/d51b7357c7ba4e22ae5f3425b098a380 deleted
/home/work/model/multi_input_model/mlruns/21/085bfa4d4a034096be5ea17b1b3939a6 active
/home/work/model/multi_input_model/mlruns/21/085cad13d09

/home/work/model/multi_input_model/mlruns/19/7b38bf8a7887496aa915e763002527a1 deleted
/home/work/model/multi_input_model/mlruns/19/ee1e0a265f3d4b94a5f0e38b805bdbb2 deleted
/home/work/model/multi_input_model/mlruns/19/64538262cbc74883b0428dd8123e0143 deleted
/home/work/model/multi_input_model/mlruns/19/20045c6374c04a20aa7c43ffc8eff356 deleted
/home/work/model/multi_input_model/mlruns/19/e3ac0e1ed2cd4746acb4ad2b29fc3fcc active
/home/work/model/multi_input_model/mlruns/19/d70d24db65634ad4b5376028def6359c deleted
/home/work/model/multi_input_model/mlruns/19/7b53e565f1fd49b48a71d1593bff961d deleted
/home/work/model/multi_input_model/mlruns/19/f1f4739029754610919200657fbf7803 deleted
/home/work/model/multi_input_model/mlruns/19/0c56b56f2d9e4831985c52fc7eff2962 active
/home/work/model/multi_input_model/mlruns/19/14a3d40a70ea4f3ca1d632df07ab7d48 deleted
/home/work/model/multi_input_model/mlruns/19/fce12152007e41d9b0228b606121f3d3 active
/home/work/model/multi_input_model/mlruns/19/aa1026a8da39

/home/work/model/multi_input_model/mlruns/6/a2fc4a33f63245f6a794af6e51a6b37a active
/home/work/model/multi_input_model/mlruns/6/53d190f36d92418e9a9b6bc4e2c075a7 deleted
/home/work/model/multi_input_model/mlruns/6/07b850304f8944a1bdabbd62d46b85d8 active
/home/work/model/multi_input_model/mlruns/6/52c5e4724165477ea5b4642380480326 active
/home/work/model/multi_input_model/mlruns/6/54074feeac1849dba97d40b5d3deb418 active
/home/work/model/multi_input_model/mlruns/6/717c8bf2355c4473b41755bcfcb43c23 active
/home/work/model/multi_input_model/mlruns/6/67e590a0a55640c2ab3c969a63c795e1 active
/home/work/model/multi_input_model/mlruns/6/6d76987d2cfe4f87af601ba4935c5167 active
/home/work/model/multi_input_model/mlruns/6/a15e3d505dc846b9ae8564775670e5f8 active
/home/work/model/multi_input_model/mlruns/6/6cc3cfaba9054b3aa90652220e571fcf active
/home/work/model/multi_input_model/mlruns/6/855a9757e0d845dd8636fa20ec2a36ba active
/home/work/model/multi_input_model/mlruns/6/03a8783813a24de793f10259fecb9ba

/home/work/model/multi_input_model/mlruns/2/f330f6318987489aa7f8010d1d592bc9 deleted
/home/work/model/multi_input_model/mlruns/2/91fd947244e4413c8d040c93886c249f deleted
/home/work/model/multi_input_model/mlruns/2/37fc481c38d74cb4bf2bdb669ea579f3 deleted
/home/work/model/multi_input_model/mlruns/2/879dd4dd35764b9baaa0ee6bce95aa40 deleted
/home/work/model/multi_input_model/mlruns/2/5cfe59761a5f46999700c85b6ed4557f deleted
/home/work/model/multi_input_model/mlruns/2/862ff3a547c7472492ef2a269397f4a4 active
/home/work/model/multi_input_model/mlruns/2/b060499298d142a497db6dd703aa70f9 deleted
/home/work/model/multi_input_model/mlruns/2/615df7baf21b455687f4592313ba3023 active
/home/work/model/multi_input_model/mlruns/2/76cbd856588e480b953a7fa15e63b3ee deleted
/home/work/model/multi_input_model/mlruns/2/b474ffa307b84827841e22801b4b1f32 deleted
/home/work/model/multi_input_model/mlruns/2/9903029895324b76a16f339a234c21d9 active
/home/work/model/multi_input_model/mlruns/2/452e9bd623224901959694db

/home/work/model/multi_input_model/mlruns/37/d2c79017ed80481c9256cb66d28a6e79 active
/home/work/model/multi_input_model/mlruns/37/a13531bf6fca4df0ba29afca45e3aafb active
/home/work/model/multi_input_model/mlruns/37/77e185019b1749bdb4d55d51d3cfe00c active
/home/work/model/multi_input_model/mlruns/37/8d6e5545dab9424cb44583990cfa6408 active
/home/work/model/multi_input_model/mlruns/37/90dc67fdc659434e9d7602c2a59fb536 active
/home/work/model/multi_input_model/mlruns/37/31499cf8b80144b9a758fb5313fd0b9b active
/home/work/model/multi_input_model/mlruns/37/b70d99d04b074ed49a90bd83482cdf83 active
/home/work/model/multi_input_model/mlruns/37/ee24bb6a9cc44e22bc4c81a58fa8cf53 active
/home/work/model/multi_input_model/mlruns/37/9b0171d606874adfb8177c274185de7b active
/home/work/model/multi_input_model/mlruns/37/952afbbe9e6c4c6684ec6bdf06bde032 active
/home/work/model/multi_input_model/mlruns/37/7c02d2566c2b46af9dd1e7a24cd0fdff active
/home/work/model/multi_input_model/mlruns/37/0806950348454b699bed

/home/work/model/multi_input_model/mlruns/29/d51090bfd4f7479bb2a028c2fa881596 deleted
/home/work/model/multi_input_model/mlruns/29/7e712c95a97640828cde1e4c6de28fc7 deleted
/home/work/model/multi_input_model/mlruns/29/90f1edee60af450d9e3d561b97dc26e7 deleted
/home/work/model/multi_input_model/mlruns/29/0a27f638f19b427899d8157712a040dd active
/home/work/model/multi_input_model/mlruns/29/01ff15bac5cb479995bf2d220c6f6c5b active
/home/work/model/multi_input_model/mlruns/29/a32abc7020764e6b96cd8cc56ff435f6 deleted
/home/work/model/multi_input_model/mlruns/29/eeb7386841cb4b35920923b94f91620d deleted
/home/work/model/multi_input_model/mlruns/29/39162e8bac534250a8dbde327717d831 active
/home/work/model/multi_input_model/mlruns/29/3952e182cd254ab4b692cf088a5c5b79 active
/home/work/model/multi_input_model/mlruns/29/5ea1ee76f9e64de6958a841a420a09b2 active
/home/work/model/multi_input_model/mlruns/29/de09063a5b6a4df192e90d327e242716 deleted
/home/work/model/multi_input_model/mlruns/29/b9ff0eb7a6c346